In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import shutil
import seaborn as sns
import sklearn, os, time, h5py
import SimpleITK as sitk
import matplotlib.pyplot as plt
from random import randint
import random
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Dropout, LeakyReLU
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, train_test_split
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from math import floor
from tensorflow.keras.applications import InceptionV3, DenseNet121
from bayes_opt import BayesianOptimization
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
# os.environ["CUDA_VISIBLE_DEVICES"]='2'

C:\Users\USER\anaconda3\envs\tensorflowgpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\USER\anaconda3\envs\tensorflowgpu\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\USER\anaconda3\envs\tensorflowgpu\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\USER\anaconda3\envs\tensorflowgpu\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
root_path = 'C:/Users/USER/Downloads/anhi/'
import zipfile
with zipfile.ZipFile(root_path + "Dataset.zip","r") as zip_ref:
    zip_ref.extractall(root_path + "")

datasetFolderName = root_path + 'Dataset'
sourceFiles = []
classLabels = ['light', 'moderate', 'no', 'severe']
X = []
Y = []

img_rows, img_cols = 224, 224 
train_path = datasetFolderName+'/train/'
validation_path = datasetFolderName+'/valid/'
test_path = datasetFolderName+'/test/'

def transferBetweenFolders(source, dest, splitRate): 
    global sourceFiles
    sourceFiles = os.listdir(source)
    if(len(sourceFiles)!=0):
        transferFileNumbers = int(len(sourceFiles)*splitRate)
        transferIndex = random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source+str(sourceFiles[eachIndex]), dest+str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")
        
def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(datasetFolderName+'/'+source+'/'+label+'/', 
                               datasetFolderName+'/'+dest+'/'+label+'/', splitRate)
                               
def my_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1Score = f1_score(y_true, y_pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("recall : {}".format(recall))
    print("f1Score : {}".format(f1Score))
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, recall, f1Score

def prepareNameWithLabels(folderName):
    sourceFiles=os.listdir(datasetFolderName+'/train/'+folderName)
    for val in sourceFiles:
        X.append(val)
        for i in range(len(classLabels)):
            if(folderName==classLabels[i]):
                Y.append(i)
for i in range(len(classLabels)):
    prepareNameWithLabels(classLabels[i])
    
X = np.asarray(X)
Y = np.asarray(Y)
IMAGE_SIZE = [224, 224]

In [4]:
# Create function
def nn_cl_bo(neurons1, neurons2, activation, optimizer, learning_rate, epochs, batch_size, dropout_rate, 
             layers1, layers2, dropout):
    optimizerL = ['SGD', 'Adam', 'RMSprop']
    optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate), 'RMSprop':RMSprop(lr=learning_rate)}               
    activationL = ['relu', 'sigmoid', 'selu', 'tanh', 'elu']

    neurons1 = round(neurons1)  
    neurons2 = round(neurons2)
    activation = activationL[round(activation)]
    optimizer = (optimizerD[optimizerL[round(optimizer)]])
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def CNN_model():
        mobilenet = MobileNet(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
        for layer in mobilenet.layers:
            layer.trainable = False
        x = Flatten()(mobilenet.output)
        
        for i in range(layers1):
            x = Dense(neurons1, activation = activation)(x)
        if dropout > 0.5:
            x = Dropout(dropout_rate)(x)
        for i in range(layers2):
            x = Dense(neurons2, activation = activation)(x)

        prediction = Dense(len(classLabels), activation='softmax')(x)
        model = Model(inputs=mobilenet.input, outputs=prediction)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])       
        return model

    skf = StratifiedKFold(n_splits=5, shuffle=True)
    skf.get_n_splits(X, Y)
    foldNum = 0
    for train_index, val_index in skf.split(X, Y):
        transferAllClassBetweenFolders('valid', 'train', 1.0)
        foldNum += 1
        X_train, X_val = X[train_index], X[val_index]
        Y_train, Y_val = Y[train_index], Y[val_index]
        for eachIndex in range(len(X_val)):
            classLabel = ''
            for i in range(len(classLabels)):
                if(Y_val[eachIndex]==i):
                    classLabel=classLabels[i]
            shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex], 
                        datasetFolderName+'/valid/'+classLabel+'/'+X_val[eachIndex])   
            
        train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
        validation_datagen = ImageDataGenerator(rescale=1./255)
        test_datagen = ImageDataGenerator(rescale=1./255)

        train_generator = train_datagen.flow_from_directory(
            train_path,target_size=(img_rows, img_cols),
            batch_size=batch_size, class_mode='categorical', subset='training')     

        validation_generator = validation_datagen.flow_from_directory(
            validation_path,  target_size=(img_rows, img_cols),
            batch_size=batch_size, class_mode='categorical')   

        test_generator = test_datagen.flow_from_directory(
            test_path, target_size=(img_rows, img_cols),
            batch_size=batch_size,  class_mode=None, shuffle=False) 
    
    
    model1=CNN_model()
    history=model1.fit_generator(train_generator, validation_data=validation_generator, epochs=epochs)

    predictions = model1.predict(test_generator, verbose=1)
    yPredictions = np.argmax(predictions, axis=1)
    true_classes = test_generator.classes
    
    score = accuracy_score(true_classes, yPredictions)
    return score

In [5]:
import time
start=time.time()
# Set paramaters
params_nn ={
    'neurons1': (64, 512),
    'neurons2': (64, 512),
    'activation':(0, 4),
    'optimizer':(0,2),
    'learning_rate':(1e-4, 1e-2),
    'batch_size':(8 ,64),
    'epochs':(30,100),
    'layers1':(1,2),
    'layers2':(1,2),
    'dropout': (0,1),
    'dropout_rate': (0, 0.3)
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=111)
#n_iter: how many steps of Bayesian optimization you want to perform. 
#The more step the more likely to find a good maximum you are
#init_points: How many teps of random exploration you want to perform. 
#Random exploration can help by diversifying the exploration space. 
nn_bo.maximize(init_points=10 , n_iter=4)
print('It takes %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  layers1  |  layers2  | learni... | neurons1  | neurons2  | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
Found 1600 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
Found 1600 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
Found 1600 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
Found 1600 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
Found 1600 images belonging to 4 classes.
Found 400 images belonging to 

In [6]:
print(nn_bo.max)

{'target': 0.8, 'params': {'activation': 0.020286305373350633, 'batch_size': 20.91169181452548, 'dropout': 0.1110226980489678, 'dropout_rate': 0.2652425035358042, 'epochs': 51.29466799425829, 'layers1': 1.6853542078693775, 'layers2': 1.7650189535072887, 'learning_rate': 0.00404963396222202, 'neurons1': 188.9195948518181, 'neurons2': 390.7419657419016, 'optimizer': 0.12452501756925849}}


In [7]:
params_nn_ = nn_bo.max['params']
learning_rate = params_nn_['learning_rate']
activationL = ['relu', 'sigmoid', 'selu', 'tanh', 'elu']
params_nn_['activation'] = activationL[round(params_nn_['activation'])]
params_nn_['batch_size'] = round(params_nn_['batch_size'])
params_nn_['epochs'] = round(params_nn_['epochs'])
params_nn_['layers1'] = round(params_nn_['layers1'])
params_nn_['layers2'] = round(params_nn_['layers2'])
params_nn_['neurons1'] = round(params_nn_['neurons1'])
params_nn_['neurons2'] = round(params_nn_['neurons2'])
optimizerL = ['Adam', 'SGD', 'RMSprop']
optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate), 'RMSprop':RMSprop(lr=learning_rate)}           
params_nn_['optimizer'] = optimizerD[optimizerL[round(params_nn_['optimizer'])]]
params_nn_

{'activation': 'relu',
 'batch_size': 21,
 'dropout': 0.1110226980489678,
 'dropout_rate': 0.2652425035358042,
 'epochs': 51,
 'layers1': 2,
 'layers2': 2,
 'learning_rate': 0.00404963396222202,
 'neurons1': 189,
 'neurons2': 391,
 'optimizer': <tensorflow.python.keras.optimizer_v2.adam.Adam at 0x139351018b0>}

In [ ]:
# def CNN_model():
#     mobilenet = MobileNet(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#     for layer in mobilenet.layers:
#         layer.trainable = False
#     x = Flatten()(mobilenet.output)

#     for i in range(layers1):
#         x = Dense(neurons1, activation = activation)(x)
#     if dropout > 0.5:
#         x = Dropout(dropout_rate)(x)
#     for i in range(layers2):
#         x = Dense(neurons2, activation = activation)(x)

#     prediction = Dense(len(classLabels), activation='softmax')(x)
#     model = Model(inputs=mobilenet.input, outputs=prediction)
#     model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])       
#     return model

In [ ]:
# skf = StratifiedKFold(n_splits=5, shuffle=True)
# skf.get_n_splits(X, Y)
# foldNum = 0
# for train_index, val_index in skf.split(X, Y):
#     transferAllClassBetweenFolders('valid', 'train', 1.0)
#     foldNum += 1
#     X_train, X_val = X[train_index], X[val_index]
#     Y_train, Y_val = Y[train_index], Y[val_index]
#     for eachIndex in range(len(X_val)):
#         classLabel = ''
#         for i in range(len(classLabels)):
#             if(Y_val[eachIndex]==i):
#                 classLabel=classLabels[i]
#         shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex], 
#                     datasetFolderName+'/valid/'+classLabel+'/'+X_val[eachIndex])   

#     train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
#     validation_datagen = ImageDataGenerator(rescale=1./255)
#     test_datagen = ImageDataGenerator(rescale=1./255)

#     train_generator = train_datagen.flow_from_directory(
#         train_path,target_size=(img_rows, img_cols),
#         batch_size=batch_size, class_mode='categorical', subset='training')     

#     validation_generator = validation_datagen.flow_from_directory(
#         validation_path,  target_size=(img_rows, img_cols),
#         batch_size=batch_size, class_mode='categorical')   

#     test_generator = test_datagen.flow_from_directory(
#         test_path, target_size=(img_rows, img_cols),
#         batch_size=batch_size,  class_mode=None, shuffle=False) 


# model1=CNN_model()
# history=model1.fit_generator(train_generator, validation_data=validation_generator, epochs=epochs)

# print("==============TEST RESULTS============") 
# predictions = model1.predict(test_generator, verbose=1)
# yPredictions = np.argmax(predictions, axis=1)
# true_classes = test_generator.classes

# testAcc, testPrec, testRecall, testFScore = my_metrics(true_classes, yPredictions)

In [ ]:
#https://www.analyticsvidhya.com/blog/2021/05/bayesian-optimization-bayes_opt-or-hyperopt/
#https://machinelearningapplied.com/hyperparameter-search-with-bayesian-optimization-for-keras-cnn-classification-and-ensembling/
#https://towardsdatascience.com/tune-deep-neural-networks-using-bayesian-optimization-c9f6503a049f

In [ ]:
#https://towardsdatascience.com/tune-deep-neural-networks-using-bayesian-optimization-c9f6503a049f
# https://www.kaggle.com/code/ishivinal/hyperparamters-optimization-gs-rs-boa-tpe-hb-ga